In [0]:
!apt update -qq;
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub;
!apt-get update -qq;
!apt-get install cuda gcc-4.8 g++-4.8 -y -qq;
!apt install cuda-8.0;
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin
!rm /usr/local/cuda/bin/g++;
!rm /usr/local/cuda/bin/gcc;
!ln -s /usr/bin/gcc-4.8 /usr/local/cuda/bin/gcc;
!ln -s /usr/bin/g++-4.8 /usr/local/cuda/bin/g++;

62 packages can be upgraded. Run 'apt list --upgradable' to see them.
--2019-01-20 13:10:37--  https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.162.216
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.162.216|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?MdN1PmflH8iEIH60m4T9JavDaophjhXThqRJZ8tCni7WeCt10lMq9ECHRgFjHC9ekkGIrWPS05-u2bXsrYMw2yxpV2cNrxlCmI_D2rymYE-3x6dqoj4cHQl4DGYLx6YX-9PoCWIi8esS2XWKe7tRjpzsJUqGU9Ra2Fg1c3DvMU3zB_C0t4oaTQyFmopGAiPIxO-lcBdU0bfSD-SRrfhdiWcqGw [following]
--2019-01-20 13:10:37--  https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?MdN1PmflH8iEIH60m4T9Ja

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%%cu
#include <iostream>


#include <stdio.h>
#include <stdlib.h>
#include <fcntl.h>
#include "string.h"


#define DEFAULT_THRESHOLD  4000

#define DEFAULT_FILENAME "/content/drive/My Drive/Colab Notebooks/BWstop-sign.ppm"//baboon.pgm"//




unsigned int *read_ppm( char *filename, int * xsize, int * ysize, int *maxval ){
  
  if ( !filename || filename[0] == '\0') {
    fprintf(stderr, "read_ppm but no file name\n");
    return NULL;  // fail
  }

  FILE *fp;

  fprintf(stderr, "read_ppm( %s )\n", filename);
  fp = fopen( filename, "rb");
  if (!fp) 
    {
      fprintf(stderr, "read_ppm()    ERROR  file '%s' cannot be opened for reading\n", filename);
      return NULL; // fail 

    }

  char chars[1024];
  //int num = read(fd, chars, 1000);
  int num = fread(chars, sizeof(char), 1000, fp);

  if (chars[0] != 'P' || chars[1] != '6') 
    {
      fprintf(stderr, "Texture::Texture()    ERROR  file '%s' does not start with \"P6\"  I am expecting a binary PPM file\n", filename);
      return NULL;
    }

  unsigned int width, height, maxvalue;


  char *ptr = chars+3; // P 6 newline
  if (*ptr == '#') // comment line! 
    {
      ptr = 1 + strstr(ptr, "\n");
    }

  num = sscanf(ptr, "%d\n%d\n%d",  &width, &height, &maxvalue);
  fprintf(stderr, "read %d things   width %d  height %d  maxval %d\n", num, width, height, maxvalue);  
  *xsize = width;
  *ysize = height;
  *maxval = maxvalue;
  
  unsigned int *pic = (unsigned int *)malloc( width * height * sizeof(unsigned int));
  if (!pic) {
    fprintf(stderr, "read_ppm()  unable to allocate %d x %d unsigned ints for the picture\n", width, height);
    return NULL; // fail but return
  }

  // allocate buffer to read the rest of the file into
  int bufsize =  3 * width * height * sizeof(unsigned char);
  if ((*maxval) > 255) bufsize *= 2;
  unsigned char *buf = (unsigned char *)malloc( bufsize );
  if (!buf) {
    fprintf(stderr, "read_ppm()  unable to allocate %d bytes of read buffer\n", bufsize);
    return NULL; // fail but return
  }





  // TODO really read
  char duh[80];
  char *line = chars;

  // find the start of the pixel data.   no doubt stupid
  sprintf(duh, "%d\0", *xsize);
  line = strstr(line, duh);
  //fprintf(stderr, "%s found at offset %d\n", duh, line-chars);
  line += strlen(duh) + 1;

  sprintf(duh, "%d\0", *ysize);
  line = strstr(line, duh);
  //fprintf(stderr, "%s found at offset %d\n", duh, line-chars);
  line += strlen(duh) + 1;

  sprintf(duh, "%d\0", *maxval);
  line = strstr(line, duh);


  fprintf(stderr, "%s found at offset %d\n", duh, line - chars);
  line += strlen(duh) + 1;

  long offset = line - chars;
  //lseek(fd, offset, SEEK_SET); // move to the correct offset
  fseek(fp, offset, SEEK_SET); // move to the correct offset
  //long numread = read(fd, buf, bufsize);
  long numread = fread(buf, sizeof(char), bufsize, fp);
  fprintf(stderr, "Texture %s   read %ld of %ld bytes\n", filename, numread, bufsize); 

  fclose(fp);


  int pixels = (*xsize) * (*ysize);
  for (int i=0; i<pixels; i++) pic[i] = (int) buf[3*i];  // red channel

 

  return pic; // success
}





void write_ppm( char *filename, int xsize, int ysize, int maxval, int *pic) 
{
  FILE *fp;
  int x,y;
  
  fp = fopen(filename, "w");
  if (!fp) 
    {
      fprintf(stderr, "FAILED TO OPEN FILE '%s' for writing\n");
      exit(-1); 
    }
  
  
  
  fprintf(fp, "P6\n"); 
  fprintf(fp,"%d %d\n%d\n", xsize, ysize, maxval);
  
  int numpix = xsize * ysize;
  for (int i=0; i<numpix; i++) {
    unsigned char uc = (unsigned char) pic[i];
    fprintf(fp, "%c%c%c", uc, uc, uc); 
  }
  fclose(fp);

}




void HostPrewitt(int xsize, int ysize,int thresh, unsigned int *input_pic, int *output)
{
    int i, j, magnitude, sum1, sum2;

    for (i = 1;  i < ysize - 1; i++) {
      for (j = 1; j < xsize -1; j++) {

        int offset = i*xsize + j;

        sum1 =  input_pic[ xsize * (i-1) + j+1 ] -     input_pic[ xsize*(i-1) + j-1 ] 
          +     input_pic[ xsize * (i)   + j+1 ] -     input_pic[ xsize*(i)   + j-1 ]
          +     input_pic[ xsize * (i+1) + j+1 ] -     input_pic[ xsize*(i+1) + j-1 ];

        sum2 = input_pic[ xsize * (i-1) + j-1 ] +     input_pic[ xsize * (i-1) + j ]  + input_pic[ xsize * (i-1) + j+1 ]
              - input_pic[xsize * (i+1) + j-1 ] -     input_pic[ xsize * (i+1) + j ] - input_pic[ xsize * (i+1) + j+1 ];

        magnitude =  sum1*sum1 + sum2*sum2;

        if (magnitude > thresh)
          output[offset] = 255;
        else 
          output[offset] = 0;
      }
    }
}


__global__ void gpuPrewittWithoutSm(int xsize, int ysize,int thresh, unsigned int *input_pic, int *result)
{
    int bx = blockIdx.x;  int by = blockIdx.y;
    int tx = threadIdx.x; int ty = threadIdx.y;
    
    int tile_w = 5;

    int Row = by * tile_w + ty;
    int Col = bx * tile_w + tx;
    
    if((Row > 0) && (Row < ysize-1)&&(Col > 0) && (Col < xsize-1))
    {
        int offset = Row*xsize + Col;
        int magnitude, sum1, sum2;

        sum1 =  input_pic[ xsize * (Row-1) + Col+1 ] -     input_pic[ xsize*(Row-1) + Col-1 ] 
          +     input_pic[ xsize * (Row)   + Col+1 ] -     input_pic[ xsize*(Row)   + Col-1 ]
          +     input_pic[ xsize * (Row+1) + Col+1 ] -     input_pic[ xsize*(Row+1) + Col-1 ];

        sum2 = input_pic[ xsize * (Row-1) + Col-1 ] +     input_pic[ xsize * (Row-1) + Col ]  + input_pic[ xsize * (Row-1) + Col+1 ]
              - input_pic[xsize * (Row+1) + Col-1 ] -     input_pic[ xsize * (Row+1) + Col ] - input_pic[ xsize * (Row+1) + Col+1 ];

        magnitude =  sum1*sum1 + sum2*sum2;

        if (magnitude > thresh)
          result[offset] = 255;
        else 
          result[offset] = 0;
    }

}


__global__ void gpuPrewittWITHSm(int xsize, int ysize,int thresh, unsigned int *input_pic, int *result)
{
    const int tile_w = 5;
    
    __shared__ int blk_share[tile_w+2][tile_w+2];
    
    int bx = blockIdx.x;  int by = blockIdx.y;
    int tx = threadIdx.x; int ty = threadIdx.y;

    int Row = by * tile_w + ty;
    int Col = bx * tile_w + tx;
    
    blk_share[ty+1][tx+1] = input_pic[ xsize * (Row) + Col ];
    
    if((ty == 0) && (Row > 0))
    {
        blk_share[ty][tx+1] = input_pic[ xsize * (Row-1) + Col ];
    }
    else if((ty == tile_w-1) && (Row < ysize-1))
    {
        blk_share[ty+2][tx+1] = input_pic[ xsize * (Row+1) + Col ];
    }
    
    if((tx==0)&&(Col>0))
    {
        blk_share[ty+1][tx] = input_pic[ xsize * (Row) + Col-1 ];
    }
    else if((tx == tile_w-1) && (Col < xsize-1))
    {
        blk_share[ty+1][tx+2] = input_pic[ xsize * (Row) + Col+1 ];
    }
    
    if((tx==1) && (ty==1)&&(Col-2>=0)&&(Row-2>=0))
    {
        blk_share[0][0] = input_pic[ xsize * (Row-2) + Col-2 ];
    }
    else if((tx==tile_w-2) && (ty==tile_w-2)&&(Row+2 <=ysize-1) && (Col+2 <= xsize-1))
    {
        blk_share[tile_w+1][tile_w+1] = input_pic[ xsize * (Row+2) + Col+2 ];
    }
    else if((tx ==tile_w-2) && (ty==1)&&(Row-2>=0)&&(Col+2 <= xsize-1))
    {
        blk_share[0][tile_w+1] = input_pic[ xsize * (Row-2) + Col+2 ];
    }
    else if((tx ==1) && (ty==tile_w-2)&&(Row+2 <=ysize-1)&&(Col-2>=0))
    {
        blk_share[tile_w+1][0] = input_pic[ xsize * (Row+2) + Col-2 ];
    }
    
    __syncthreads();
    
    
    if((Row > 0) && (Row < ysize-1)&&(Col > 0) && (Col < xsize-1))
    {
        int offset = Row*xsize + Col;
        int magnitude, sum1, sum2;
        
        ty++;
        tx++;
        
       {
          sum1 =  blk_share[ty-1][tx+1] -     blk_share[ty-1][tx-1]  
            +     blk_share[ty][tx+1]   -     blk_share[ty][tx-1]
            +     blk_share[ty+1][tx+1] -     blk_share[ty+1][tx-1];

          sum2 =  blk_share[ty-1][tx-1] +     blk_share[ty-1][tx]  + blk_share[ty-1][tx+1]
                - blk_share[ty+1][tx-1] -     blk_share[ty+1][tx]  - blk_share[ty+1][tx+1];
        }

        magnitude =  sum1*sum1 + sum2*sum2;

        if (magnitude > thresh)
          result[offset] = 255;
        else 
          result[offset] = 0;
    }
    __syncthreads();

}


void GPUPrewittHandler(int xsize, int ysize,int thresh, unsigned int *input_pic, int *output)
{
    int size = xsize * ysize * sizeof(unsigned int);
    
    unsigned int *pic;
    int *result;
    int tile_w = 5;
    
    cudaMalloc(&pic, size);
    cudaMemcpy(pic, input_pic, size, cudaMemcpyHostToDevice);
    
    cudaMalloc(&result, size);
    
    dim3 DimGrid(ceil(xsize/tile_w), ceil(ysize/tile_w), 1);
    dim3 DimBlock(5, 5, 1);

    
   // gpuPrewittWithoutSm<<<DimGrid, DimBlock>>>(xsize, ysize, thresh, pic, result);
    gpuPrewittWITHSm<<<DimGrid, DimBlock>>>(xsize, ysize, thresh, pic, result);
    
    
    cudaMemcpy(output, result, size, cudaMemcpyDeviceToHost);
    
    // Free device matrices
    cudaFree(pic); cudaFree(result);

    
}


int main( int argc, char **argv )
{

  int thresh = DEFAULT_THRESHOLD;
  char *filename;
  filename = strdup( DEFAULT_FILENAME);
  
  if (argc > 1) {
    if (argc == 3)  { // filename AND threshold
      filename = strdup( argv[1]);
       thresh = atoi( argv[2] );
    }
    if (argc == 2) { // default file but specified threshhold
      
      thresh = atoi( argv[1] );
    }

    fprintf(stderr, "file %s    threshold %d\n", filename, thresh); 
  }


  int xsize, ysize, maxval;
  unsigned int *pic = read_ppm( filename, &xsize, &ysize, &maxval ); 


  int numbytes =  xsize * ysize *  sizeof( int );//3 * sizeof( int );
  int *result = (int *) malloc( numbytes );
  if (!result) { 
    fprintf(stderr, "prewitt() unable to malloc %d bytes\n", numbytes);
    exit(-1); // fail
  }

  int i, j, magnitude, sum1, sum2; 
  int *out = result;

  for (int col=0; col<ysize; col++) {
    for (int row=0; row<xsize; row++) { 
      *out++ = 0; 
    }
  }

  
  //HostPrewitt(xsize, ysize,thresh, pic, result);
  GPUPrewittHandler(xsize, ysize, thresh, pic, result);
    
  write_ppm( "/content/drive/My Drive/Colab Notebooks/result.ppm", xsize, ysize, 255, result);

  fprintf(stderr, "prewitt done\n"); 

}

'read_ppm( /content/drive/My Drive/Colab Notebooks/BWstop-sign.ppm )\nread 3 things   width 384  height 480  maxval 255\n255 found at offset 50\nTexture /content/drive/My Drive/Colab Notebooks/BWstop-sign.ppm   read 552960 of 552960 bytes\nprewitt done\n'